In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/3a-superstore/Branches_ENG.csv
/kaggle/input/3a-superstore/Categories.csv
/kaggle/input/3a-superstore/Categories_ENG.csv
/kaggle/input/3a-superstore/Orders.csv
/kaggle/input/3a-superstore/Customers_ENG.csv
/kaggle/input/3a-superstore/Customers.csv
/kaggle/input/3a-superstore/Order_Details.csv
/kaggle/input/3a-superstore/Branches.csv


In [2]:
import polars as pl

# Orders (chỉ lấy cột cần thiết)
orders = (
    pl.scan_csv("/kaggle/input/3a-superstore/Orders.csv", try_parse_dates=True)
    .select(["USERID", "ORDERID", "BRANCH_ID", "TOTALBASKET", "DATE_"])
)

# Order Details - đảm bảo (ORDERID, ITEMID) là duy nhất
details = (
    pl.scan_csv("/kaggle/input/3a-superstore/Order_Details.csv")
    .select(["ORDERID", "ITEMID", "TOTALPRICE"])
    .unique()
)

# Categories (file .csv có dấu ';') - Ensure ITEMID is unique
categories = (
    pl.read_csv("/kaggle/input/3a-superstore/Categories_ENG.csv", separator=';')
    .select(["ITEMID", "CATEGORY1","ITEMNAME"])        
    .unique(subset=["ITEMID"])
)

# Branches (file .csv có dấu ';') - Ensure BRANCH_ID is unique
branches = (
    pl.read_csv("/kaggle/input/3a-superstore/Branches_ENG.csv", separator=';')
    .select(["BRANCH_ID", "REGION"])
    .unique(subset=["BRANCH_ID"])
)

# Gộp dữ liệu (vẫn dùng lazy nên ít tốn RAM)
merged = (
    orders
    .join(details, on="ORDERID", how="left")
    .join(categories.lazy(), on="ITEMID", how="left")
    .join(branches.lazy(), on="BRANCH_ID", how="left")
    .select(["USERID", "ORDERID", "ITEMID", "TOTALPRICE", "TOTALBASKET", "CATEGORY1","ITEMNAME", "REGION", "DATE_"])
    .collect(engine="streaming")
)

# Kiểm tra
print(merged.head())
print(f"✅ Tổng số dòng sau khi gộp: {merged.shape[0]:,}")


shape: (5, 9)
┌────────┬─────────┬────────┬─────────────┬───┬─────────────┬────────────┬────────────┬────────────┐
│ USERID ┆ ORDERID ┆ ITEMID ┆ TOTALPRICE  ┆ … ┆ CATEGORY1   ┆ ITEMNAME   ┆ REGION     ┆ DATE_      │
│ ---    ┆ ---     ┆ ---    ┆ ---         ┆   ┆ ---         ┆ ---        ┆ ---        ┆ ---        │
│ i64    ┆ i64     ┆ i64    ┆ str         ┆   ┆ str         ┆ str        ┆ str        ┆ datetime[μ │
│        ┆         ┆        ┆             ┆   ┆             ┆            ┆            ┆ s]         │
╞════════╪═════════╪════════╪═════════════╪═══╪═════════════╪════════════╪════════════╪════════════╡
│ 40955  ┆ 1201619 ┆ 26305  ┆ 865,6200000 ┆ … ┆ Breakfast   ┆ PINAR      ┆ Guneydogu  ┆ 2022-12-21 │
│        ┆         ┆        ┆ 000001      ┆   ┆ Products    ┆ CHEESE 500 ┆ Anadolu    ┆ 00:00:00   │
│        ┆         ┆        ┆             ┆   ┆             ┆ GR SLICED  ┆            ┆            │
│        ┆         ┆        ┆             ┆   ┆             ┆ TOA…       ┆   

In [3]:
# Kiểm tra kiểu dữ liệu
print(merged.dtypes)

# Kiểm tra các giá trị null
print(merged.null_count())

# Kiểm tra thống kê cơ bản cho cột số
print(merged.select(["TOTALPRICE", "TOTALBASKET"]).describe())


[Int64, Int64, Int64, String, String, String, String, String, Datetime(time_unit='us', time_zone=None)]
shape: (1, 9)
┌────────┬─────────┬────────┬────────────┬───┬───────────┬──────────┬──────────┬───────┐
│ USERID ┆ ORDERID ┆ ITEMID ┆ TOTALPRICE ┆ … ┆ CATEGORY1 ┆ ITEMNAME ┆ REGION   ┆ DATE_ │
│ ---    ┆ ---     ┆ ---    ┆ ---        ┆   ┆ ---       ┆ ---      ┆ ---      ┆ ---   │
│ u32    ┆ u32     ┆ u32    ┆ u32        ┆   ┆ u32       ┆ u32      ┆ u32      ┆ u32   │
╞════════╪═════════╪════════╪════════════╪═══╪═══════════╪══════════╪══════════╪═══════╡
│ 0      ┆ 0       ┆ 0      ┆ 0          ┆ … ┆ 0         ┆ 0        ┆ 15891143 ┆ 0     │
└────────┴─────────┴────────┴────────────┴───┴───────────┴──────────┴──────────┴───────┘
shape: (9, 3)
┌────────────┬───────────────────┬───────────────────┐
│ statistic  ┆ TOTALPRICE        ┆ TOTALBASKET       │
│ ---        ┆ ---               ┆ ---               │
│ str        ┆ str               ┆ str               │
╞════════════╪═══════════

# Làm sạch và tiền xử lý dữ liệu 

## Chuyển đổi kiểu dữ liệu 

In [4]:


merged = (
    merged.with_columns([
        pl.col("TOTALPRICE")
        .cast(pl.Utf8)                     # Ép sang chuỗi
        .str.replace(",", ".")             # Thay dấu phẩy
        .cast(pl.Float64)                  # Ép lại float
        .alias("TOTALPRICE"),

        pl.col("TOTALBASKET")
        .cast(pl.Utf8)
        .str.replace(",", ".")
        .cast(pl.Float64)
        .alias("TOTALBASKET"),
    ])
)
print(merged.select(["TOTALPRICE", "TOTALBASKET"]).describe())

shape: (9, 3)
┌────────────┬─────────────┬─────────────┐
│ statistic  ┆ TOTALPRICE  ┆ TOTALBASKET │
│ ---        ┆ ---         ┆ ---         │
│ str        ┆ f64         ┆ f64         │
╞════════════╪═════════════╪═════════════╡
│ count      ┆ 5.1184395e7 ┆ 5.1184395e7 │
│ null_count ┆ 0.0         ┆ 0.0         │
│ mean       ┆ 256.045207  ┆ 1621.578228 │
│ std        ┆ 548.116603  ┆ 1660.616185 │
│ min        ┆ 0.0         ┆ 0.0         │
│ 25%        ┆ 42.45       ┆ 713.28      │
│ 50%        ┆ 116.06      ┆ 1285.13     │
│ 75%        ┆ 288.12      ┆ 2108.49     │
│ max        ┆ 59520.96    ┆ 307683.45   │
└────────────┴─────────────┴─────────────┘


## Xử lý giá trị thiếu 

In [5]:
merged = (
    merged
    .with_columns([
        pl.col("REGION").fill_null("Unknown"),
    ])
)


## Xử lý ngoại lai 

In [6]:
q = merged.select([
    pl.col("TOTALPRICE").quantile(0.01).alias("p1_price"),
    pl.col("TOTALPRICE").quantile(0.99).alias("p99_price"),
    pl.col("TOTALBASKET").quantile(0.01).alias("p1_basket"),
    pl.col("TOTALBASKET").quantile(0.99).alias("p99_basket"),
]).to_dicts()[0]
merged = merged.filter(
    (pl.col("TOTALPRICE").is_between(q["p1_price"], q["p99_price"])) &
    (pl.col("TOTALBASKET").is_between(q["p1_basket"], q["p99_basket"]))
)

In [7]:
merged = merged.filter(
    (pl.col("TOTALPRICE") > 0) & 
    (pl.col("TOTALBASKET") > 0)
)


## Chuẩn hoá dữ liệu 

In [8]:
merged = (
    merged.with_columns([
        pl.col("REGION").str.strip_chars().str.to_lowercase().alias("REGION"),
        pl.col("CATEGORY1").str.strip_chars().str.to_lowercase().alias("CATEGORY1"),
        pl.col("ITEMNAME").str.strip_chars().str.to_titlecase().alias("ITEMNAME"),
    ])
)


In [9]:
print(merged.head())
print(f"✅ Sau làm sạch: {merged.shape[0]:,} dòng")
print(merged.null_count())
print(merged.select(["TOTALPRICE", "TOTALBASKET"]).describe())



shape: (5, 9)
┌────────┬─────────┬────────┬────────────┬───┬─────────────┬─────────────┬────────────┬────────────┐
│ USERID ┆ ORDERID ┆ ITEMID ┆ TOTALPRICE ┆ … ┆ CATEGORY1   ┆ ITEMNAME    ┆ REGION     ┆ DATE_      │
│ ---    ┆ ---     ┆ ---    ┆ ---        ┆   ┆ ---         ┆ ---         ┆ ---        ┆ ---        │
│ i64    ┆ i64     ┆ i64    ┆ f64        ┆   ┆ str         ┆ str         ┆ str        ┆ datetime[μ │
│        ┆         ┆        ┆            ┆   ┆             ┆             ┆            ┆ s]         │
╞════════╪═════════╪════════╪════════════╪═══╪═════════════╪═════════════╪════════════╪════════════╡
│ 40955  ┆ 1201619 ┆ 26305  ┆ 865.62     ┆ … ┆ breakfast   ┆ Pinar       ┆ guneydogu  ┆ 2022-12-21 │
│        ┆         ┆        ┆            ┆   ┆ products    ┆ Cheese 500  ┆ anadolu    ┆ 00:00:00   │
│        ┆         ┆        ┆            ┆   ┆             ┆ Gr Sliced   ┆            ┆            │
│        ┆         ┆        ┆            ┆   ┆             ┆ Toa…        ┆   

# Phân tích EDA

In [10]:
import polars as pl
import plotly.express as px

# =====================================================
# 2️⃣ Chuẩn bị dữ liệu thời gian
# =====================================================
df = merged.lazy()  

df = df.with_columns([
    pl.col("DATE_").dt.year().alias("YEAR"),
    pl.col("DATE_").dt.month().alias("MONTH"),
    pl.col("DATE_").dt.weekday().alias("WEEKDAY")
])
df = df.filter(pl.col("REGION") != "unknown")
# Chuyển sang Pandas cho Plotly
df_pd = df.collect().to_pandas()

# =====================================================
# 3️⃣ Doanh thu theo thời gian
# =====================================================
# Doanh thu theo tháng
monthly_revenue = df_pd.groupby(["YEAR", "MONTH"])["TOTALPRICE"].sum().reset_index()
fig_month = px.line(
    monthly_revenue,
    x="MONTH",
    y="TOTALPRICE",
    color="YEAR",
    title="Doanh thu theo tháng theo từng năm",
    markers=True
)
fig_month.show()

# Doanh thu theo ngày (nếu muốn chi tiết hơn)
daily_revenue = df_pd.groupby("DATE_")["TOTALPRICE"].sum().reset_index()
fig_day = px.line(
    daily_revenue,
    x="DATE_",
    y="TOTALPRICE",
    title="Doanh thu theo ngày",
    markers=False
)
fig_day.show()
#  Top 5 REGION theo doanh thu

region_revenue = (
    df_pd.groupby("REGION")["TOTALPRICE"].sum()
    .reset_index()
    .sort_values("TOTALPRICE", ascending=False)
    .head(5)
)
fig_region = px.bar(
    region_revenue,
    x="REGION",
    y="TOTALPRICE",
    title="Top 5 REGION theo doanh thu",
    text="TOTALPRICE"
)
fig_region.show()


#  Top 5 CATEGORY1 theo doanh thu

category_revenue = (
    df_pd.groupby("CATEGORY1")["TOTALPRICE"].sum()
    .reset_index()
    .sort_values("TOTALPRICE", ascending=False)
    .head(5)
)
fig_category = px.bar(
    category_revenue,
    x="CATEGORY1",
    y="TOTALPRICE",
    title="Top 5 CATEGORY1 theo doanh thu",
    text="TOTALPRICE"
)
fig_category.show()
product_revenue = (
    df_pd.groupby("ITEMNAME")["TOTALPRICE"].sum()
    .reset_index()
    .sort_values("TOTALPRICE", ascending=False)
    .head(5)
)

fig_product = px.bar(
    product_revenue,
    x="ITEMNAME",
    y="TOTALPRICE",
    title="Top 5 sản phẩm theo doanh thu",
    text="TOTALPRICE"
)
fig_product.show()
print("📅 Doanh thu theo tháng:")
print(monthly_revenue)
print("🏙️ Top 5 REGION theo doanh thu:")
print(region_revenue)
print("🛒 Top 5 CATEGORY1 theo doanh thu:")
print(category_revenue)
print("🔥 Top 5 sản phẩm theo doanh thu:")
print(product_revenue)


📅 Doanh thu theo tháng:
    YEAR  MONTH    TOTALPRICE
0   2021      1  1.705305e+08
1   2021      2  1.585629e+08
2   2021      3  1.810076e+08
3   2021      4  1.805931e+08
4   2021      5  1.922445e+08
5   2021      6  1.912677e+08
6   2021      7  2.030928e+08
7   2021      8  2.075837e+08
8   2021      9  2.052219e+08
9   2021     10  2.179335e+08
10  2021     11  2.159845e+08
11  2021     12  2.273986e+08
12  2022      1  2.563310e+08
13  2022      2  2.360267e+08
14  2022      3  2.650746e+08
15  2022      4  2.621746e+08
16  2022      5  2.740491e+08
17  2022      6  2.699044e+08
18  2022      7  2.817930e+08
19  2022      8  2.867926e+08
20  2022      9  2.807977e+08
21  2022     10  2.925817e+08
22  2022     11  2.867217e+08
23  2022     12  3.016222e+08
24  2023      1  2.874273e+08
25  2023      2  2.637838e+08
26  2023      3  2.960347e+08
27  2023      4  2.904748e+08
28  2023      5  3.019064e+08
29  2023      6  2.954817e+08
30  2023      7  2.157437e+08
31  2023      8 

In [11]:
import polars as pl
import plotly.express as px

# Lọc dữ liệu danh mục Home và loại bỏ REGION là "Unknown"
home_df = (
    merged.lazy()
    .filter(pl.col("CATEGORY1") == "home")
    # THÊM BƯỚC LỌC ĐỂ BỎ KHU VỰC "Unknown"
    .filter(pl.col("REGION") != "unknown")
    .select(["REGION", "DATE_", "TOTALPRICE"])
    .collect()
)

# Gộp theo tháng
home_monthly = (
    home_df
    .with_columns([
        pl.col("DATE_").dt.truncate("1mo").alias("MONTH")
    ])
    .group_by(["REGION", "MONTH"])
    .agg([
        pl.sum("TOTALPRICE").alias("TOTAL_REVENUE")
    ])
    .sort(["REGION", "MONTH"])
)

# Vẽ biểu đồ
fig = px.line(
    home_monthly.to_pandas(),
    x="MONTH",
    y="TOTAL_REVENUE",
    color="REGION",
    title="📈 Doanh thu danh mục Home theo khu vực (theo thời gian)",
    markers=True
)

fig.update_layout(
    xaxis_title="Thời gian (Tháng)",
    yaxis_title="Tổng doanh thu (TOTALPRICE)",
    legend_title="Khu vực (REGION)",
    template="plotly_white"
)

fig.show()

# Dự đoán doanh thu sản phẩm theo khu vực trong 6 tháng tới 

In [12]:
import polars as pl
import plotly.express as px

# Lọc dữ liệu danh mục Home và loại bỏ REGION là "Unknown"
name_df = (
    merged.lazy()
    .filter(pl.col("ITEMNAME") == "Musical Heart Plush Bear 37 Cm")
    # THÊM BƯỚC LỌC ĐỂ BỎ KHU VỰC "Unknown"
    .filter(pl.col("REGION") != "unknown")
    .select(["REGION", "DATE_", "TOTALPRICE"])
    .collect()
)

#  Gộp theo tháng
name_monthly = (
    name_df
    .with_columns([
        pl.col("DATE_").dt.truncate("1mo").alias("MONTH")
    ])
    .group_by(["REGION", "MONTH"])
    .agg([
        pl.sum("TOTALPRICE").alias("TOTAL_REVENUE")
    ])
    .sort(["REGION", "MONTH"])
)

# Vẽ biểu đồ
fig = px.line(
    name_monthly.to_pandas(),
    x="MONTH",
    y="TOTAL_REVENUE",
    color="REGION",
    title="📈 Doanh thu sản phẩm Musical Heart Plush Bear 37 Cm theo khu vực (theo thời gian)",
    markers=True
)

fig.update_layout(
    xaxis_title="Thời gian (Tháng)",
    yaxis_title="Tổng doanh thu (TOTALPRICE)",
    legend_title="Khu vực (REGION)",
    template="plotly_white"
)

fig.show()

In [13]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings("ignore")
#  Chuyển Polars DataFrame sang Pandas để dùng cho mô hình
home_monthly_pd = home_monthly.to_pandas()

#  Đảm bảo cột thời gian là kiểu datetime
home_monthly_pd["MONTH"] = pd.to_datetime(home_monthly_pd["MONTH"])


#  MÔ HÌNH XGBOOST DỰ ĐOÁN DOANH THU (6 THÁNG TỚI)


metrics_xgb = []
forecast_all = pd.DataFrame()

for region in home_monthly_pd['REGION'].unique():
    region_data = home_monthly_pd[home_monthly_pd['REGION'] == region].copy()
    region_data['MONTH'] = pd.to_datetime(region_data['MONTH'])
    region_data = region_data.sort_values('MONTH')


    #  Feature Engineering

    region_data['month'] = region_data['MONTH'].dt.month
    region_data['year'] = region_data['MONTH'].dt.year
    region_data['lag1'] = region_data['TOTAL_REVENUE'].shift(1)
    region_data['lag2'] = region_data['TOTAL_REVENUE'].shift(2)
    region_data['lag3'] = region_data['TOTAL_REVENUE'].shift(3)
    region_data['rolling_mean3'] = region_data['TOTAL_REVENUE'].rolling(window=3).mean()
    region_data = region_data.dropna().reset_index(drop=True)


    #  Train/Test Split
    split_idx = int(len(region_data) * 0.8)
    train_df = region_data.iloc[:split_idx]
    test_df = region_data.iloc[split_idx:]

    X_train = train_df[['month', 'year', 'lag1', 'lag2', 'lag3', 'rolling_mean3']]
    y_train = train_df['TOTAL_REVENUE']

    X_test = test_df[['month', 'year', 'lag1', 'lag2', 'lag3', 'rolling_mean3']]
    y_test = test_df['TOTAL_REVENUE']

    # ⚙️ Huấn luyện mô hình
    model = XGBRegressor(
        n_estimators=400,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    model.fit(X_train, y_train)


    #  Dự đoán giai đoạn test

    y_pred = model.predict(X_test)
    y_pred = np.clip(y_pred, 0, None)

    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    safe_y = np.where(y_test == 0, np.nan, y_test)
    mape = np.nanmean(np.abs((safe_y - y_pred) / safe_y)) * 100

    metrics_xgb.append({
        'REGION': region,
        'MAE': round(mae, 2),
        'RMSE': round(rmse, 2),
        'MAPE (%)': round(mape, 2)
    })


    #  Dự đoán 6 tháng tương lai

    future_preds = []
    last_data = region_data.copy()
    last_month = last_data['MONTH'].max()

    for i in range(1, 7):  # 6 tháng tới
        next_month = (last_month + pd.DateOffset(months=i)).replace(day=1)
        next_year = next_month.year
        next_m = next_month.month

        lag1 = last_data['TOTAL_REVENUE'].iloc[-1]
        lag2 = last_data['TOTAL_REVENUE'].iloc[-2]
        lag3 = last_data['TOTAL_REVENUE'].iloc[-3]
        rolling_mean3 = last_data['TOTAL_REVENUE'].iloc[-3:].mean()

        X_future = pd.DataFrame([{
            'month': next_m,
            'year': next_year,
            'lag1': lag1,
            'lag2': lag2,
            'lag3': lag3,
            'rolling_mean3': rolling_mean3
        }])

        y_future = model.predict(X_future)[0]
        y_future = max(0, y_future)

        future_preds.append({'MONTH': next_month, 'TOTAL_REVENUE': y_future, 'REGION': region})
        new_row = {'MONTH': next_month, 'TOTAL_REVENUE': y_future}
        last_data = pd.concat([last_data, pd.DataFrame([new_row])], ignore_index=True)


    #  Vẽ biểu đồ

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=region_data['MONTH'], 
        y=region_data['TOTAL_REVENUE'], 
        mode='lines+markers',
        name='Thực tế',
        line=dict(color='blue')
    ))

    fig.add_trace(go.Scatter(
        x=test_df['MONTH'], 
        y=y_pred, 
        mode='lines+markers',
        name='Dự đoán (Test)',
        line=dict(color='orange', dash='dash')
    ))

    future_df = pd.DataFrame(future_preds)
    fig.add_trace(go.Scatter(
        x=future_df['MONTH'], 
        y=future_df['TOTAL_REVENUE'],
        mode='lines+markers',
        name='Dự đoán (6 tháng tới)',
        line=dict(color='green', dash='dot')
    ))

    fig.update_layout(
        title=f"📊 Dự đoán Doanh thu Home - {region} (6 tháng tới)",
        xaxis_title="Tháng",
        yaxis_title="Tổng Doanh Thu",
        template="plotly_white",
        height=500
    )
    fig.show()

    forecast_all = pd.concat([forecast_all, future_df])


#  Bảng kết quả đánh giá

metrics_xgb_df = pd.DataFrame(metrics_xgb).sort_values("MAPE (%)")
print("📈 Độ chính xác mô hình XGBoost (Test giai đoạn gần nhất):")
display(metrics_xgb_df.style.background_gradient(subset=['MAPE (%)'], cmap='RdYlGn_r'))


#  Tổng hợp dự đoán 6 tháng tới

print("\n🔮 Dự báo 6 tháng tới cho từng khu vực:")
display(forecast_all.sort_values(["REGION", "MONTH"]))


📈 Độ chính xác mô hình XGBoost (Test giai đoạn gần nhất):


,REGION,MAE,RMSE,MAPE (%)
3,guneydogu anadolu,1087947.010000,1867335.010000,22.410000
4,ic anadolu,2294804.210000,4178751.570000,23.330000
0,akdeniz,1918381.030000,3373645.260000,23.910000
2,ege,1197390.520000,2136101.830000,24.030000
5,karadeniz,1274621.890000,2196431.220000,24.160000
1,dogu anadolu,1021808.940000,1696230.680000,24.620000
6,marmara,1618161.490000,2662955.060000,25.130000



🔮 Dự báo 6 tháng tới cho từng khu vực:


,MONTH,TOTAL_REVENUE,REGION
0,2023-09-01,13582781.0,akdeniz
1,2023-10-01,13394694.0,akdeniz
2,2023-11-01,12896722.0,akdeniz
3,2023-12-01,14082999.0,akdeniz
4,2024-01-01,13342934.0,akdeniz
5,2024-02-01,13826297.0,akdeniz
0,2023-09-01,6751636.0,dogu anadolu
1,2023-10-01,6673011.5,dogu anadolu
2,2023-11-01,6308201.5,dogu anadolu
3,2023-12-01,7069252.0,dogu anadolu


In [14]:

#  Import thư viện

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings("ignore")


#  Chuẩn bị dữ liệu

name_monthly_pd = name_monthly.to_pandas()
name_monthly_pd["MONTH"] = pd.to_datetime(name_monthly_pd["MONTH"])

# Loại bỏ outlier nhẹ
q1 = name_monthly_pd["TOTAL_REVENUE"].quantile(0.05)
q3 = name_monthly_pd["TOTAL_REVENUE"].quantile(0.95)
name_monthly_pd = name_monthly_pd[
    (name_monthly_pd["TOTAL_REVENUE"] >= q1) & 
    (name_monthly_pd["TOTAL_REVENUE"] <= q3)
].reset_index(drop=True)

metrics_xgb = []
forecast_all = pd.DataFrame()


#  Huấn luyện & Dự báo cho từng REGION

for region in name_monthly_pd['REGION'].unique():
    region_data = name_monthly_pd[name_monthly_pd['REGION'] == region].copy()
    region_data = region_data.sort_values("MONTH").reset_index(drop=True)


    #  Feature Engineering nâng cao

    region_data["month"] = region_data["MONTH"].dt.month
    region_data["year"] = region_data["MONTH"].dt.year
    region_data["month_sin"] = np.sin(2 * np.pi * region_data["month"] / 12)
    region_data["month_cos"] = np.cos(2 * np.pi * region_data["month"] / 12)
    region_data["trend"] = np.arange(len(region_data))
    region_data["trend_sq"] = region_data["trend"] ** 2

    # Log-transform để giảm độ lệch
    region_data["TOTAL_REVENUE_LOG"] = np.log1p(region_data["TOTAL_REVENUE"])

    # Lag + rolling
    for lag in [1, 2, 3, 6, 12]:
        region_data[f"lag_{lag}"] = region_data["TOTAL_REVENUE_LOG"].shift(lag)

    # Trung bình động có trọng số (EMA)
    region_data["ema_3"] = region_data["TOTAL_REVENUE_LOG"].ewm(span=3, adjust=False).mean()
    region_data["ema_6"] = region_data["TOTAL_REVENUE_LOG"].ewm(span=6, adjust=False).mean()

    region_data["roll_std_3"] = region_data["TOTAL_REVENUE_LOG"].rolling(3).std()
    region_data["roll_std_6"] = region_data["TOTAL_REVENUE_LOG"].rolling(6).std()

    region_data = region_data.dropna().reset_index(drop=True)


    #  Train/Test Split

    split_idx = int(len(region_data) * 0.8)
    train_df = region_data.iloc[:split_idx]
    test_df = region_data.iloc[split_idx:]

    X_cols = [
        "month_sin", "month_cos", "trend", "trend_sq",
        "lag_1", "lag_2", "lag_3", "lag_6", "lag_12",
        "ema_3", "ema_6", "roll_std_3", "roll_std_6"
    ]
    X_train, X_test = train_df[X_cols], test_df[X_cols]
    y_train, y_test = train_df["TOTAL_REVENUE_LOG"], test_df["TOTAL_REVENUE_LOG"]


    #  Huấn luyện XGBoost 

    model = XGBRegressor(
        n_estimators=3000,
        learning_rate=0.015,
        max_depth=8,
        subsample=0.85,
        colsample_bytree=0.85,
        reg_lambda=2.5,
        reg_alpha=0.5,
        random_state=42,
        objective='reg:squarederror',
        early_stopping_rounds=150
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        verbose=False
    )

    # Dự đoán trên tập test

    y_pred_log = model.predict(X_test)
    y_pred = np.expm1(y_pred_log)
    y_true = np.expm1(y_test)

    epsilon = 1e-10
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

    metrics_xgb.append({
        "REGION": region,
        "MAE": round(mae, 2),
        "RMSE": round(rmse, 2),
        "MAPE (%)": round(mape, 2)
    })


    #  Dự báo 6 tháng tới

    future_preds = []
    last_data = region_data.copy()

    for i in range(6):
        next_month = (last_data["MONTH"].max() + pd.DateOffset(months=1)).replace(day=1)
        next_row = {
            "month_sin": np.sin(2 * np.pi * next_month.month / 12),
            "month_cos": np.cos(2 * np.pi * next_month.month / 12),
            "trend": last_data["trend"].iloc[-1] + 1,
            "trend_sq": (last_data["trend"].iloc[-1] + 1) ** 2
        }

        for lag in [1, 2, 3, 6, 12]:
            next_row[f"lag_{lag}"] = last_data["TOTAL_REVENUE_LOG"].iloc[-lag] if len(last_data) >= lag else np.nan

        next_row["ema_3"] = last_data["TOTAL_REVENUE_LOG"].ewm(span=3, adjust=False).mean().iloc[-1]
        next_row["ema_6"] = last_data["TOTAL_REVENUE_LOG"].ewm(span=6, adjust=False).mean().iloc[-1]
        next_row["roll_std_3"] = last_data["TOTAL_REVENUE_LOG"].rolling(3).std().iloc[-1]
        next_row["roll_std_6"] = last_data["TOTAL_REVENUE_LOG"].rolling(6).std().iloc[-1]

        X_future = pd.DataFrame([next_row])[X_cols]
        y_future_log = model.predict(X_future)[0]
        y_future = np.expm1(y_future_log)

        future_preds.append({"MONTH": next_month, "TOTAL_REVENUE": y_future, "REGION": region})

        # Cập nhật dữ liệu cho vòng tiếp theo
        last_data = pd.concat([
            last_data,
            pd.DataFrame([{"MONTH": next_month, "TOTAL_REVENUE_LOG": y_future_log, "trend": next_row["trend"]}])
        ], ignore_index=True)

    future_df = pd.DataFrame(future_preds)


    #  Vẽ biểu đồ

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=region_data["MONTH"], y=np.expm1(region_data["TOTAL_REVENUE_LOG"]),
        mode="lines+markers", name="Thực tế", line=dict(color="blue")
    ))
    fig.add_trace(go.Scatter(
        x=test_df["MONTH"], y=y_pred,
        mode="lines+markers", name="Dự đoán (Test)", line=dict(color="orange", dash="dash")
    ))
    fig.add_trace(go.Scatter(
        x=future_df["MONTH"], y=future_df["TOTAL_REVENUE"],
        mode="lines+markers", name="Dự đoán (6 tháng tới)", line=dict(color="green", dash="dot")
    ))
    fig.update_layout(
        title=f" Dự đoán Doanh thu Musical Heart Plush Bear 37 Cm  - {region} (6 tháng tới)",
        xaxis_title="Tháng", yaxis_title="Tổng Doanh Thu",
        template="plotly_white", height=500
    )
    fig.show()

    forecast_all = pd.concat([forecast_all, future_df], ignore_index=True)


#  KẾT QUẢ ĐÁNH GIÁ

metrics_xgb_df = pd.DataFrame(metrics_xgb).sort_values("MAPE (%)")
print(" Độ chính xác mô hình XGBoost:")
display(metrics_xgb_df.style.background_gradient(subset=["MAPE (%)"], cmap="RdYlGn_r"))


#  TỔNG HỢP DỰ BÁO

print("\n Dự báo 6 tháng tới cho từng khu vực:")
display(forecast_all.sort_values(["REGION", "MONTH"]))


 Độ chính xác mô hình XGBoost:


,REGION,MAE,RMSE,MAPE (%)
6,marmara,5360.400000,6088.840000,11.840000
5,karadeniz,6379.470000,7985.200000,14.200000
2,ege,6194.070000,6822.420000,17.710000
1,dogu anadolu,6042.670000,6815.370000,21.320000
3,guneydogu anadolu,7894.750000,9205.880000,27.350000
0,akdeniz,14850.960000,24393.110000,64.100000
4,ic anadolu,34431.590000,42722.260000,134.340000



 Dự báo 6 tháng tới cho từng khu vực:


,MONTH,TOTAL_REVENUE,REGION
0,2023-08-01,67802.796875,akdeniz
1,2023-09-01,67266.476562,akdeniz
2,2023-10-01,67801.562500,akdeniz
3,2023-11-01,67257.304688,akdeniz
4,2023-12-01,67793.546875,akdeniz
5,2024-01-01,67793.546875,akdeniz
6,2023-07-01,36538.750000,dogu anadolu
7,2023-08-01,35006.308594,dogu anadolu
8,2023-09-01,35064.648438,dogu anadolu
9,2023-10-01,35242.000000,dogu anadolu


# Dự đoán doanh thu sản phẩm theo khu vực bằng LightGBM

In [15]:


# ==============================================================
# 📈 DỰ BÁO DOANH THU DANH MỤC HOME THEO KHU VỰC (6 THÁNG TỚI)
# ==============================================================
import polars as pl
import pandas as pd
import numpy as np
import plotly.express as px
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ==========================
# 1️⃣ LỌC DỮ LIỆU DANH MỤC HOME
# ==========================
home_df = (
    merged.lazy()
    .filter(pl.col("CATEGORY1") == "home")
    .filter(pl.col("REGION") != "unknown")
    .select(["REGION", "DATE_", "TOTALPRICE"])
    .collect()
)

# ==========================
# 2️⃣ GỘP THEO THÁNG
# ==========================
home_monthly = (
    home_df
    .with_columns([
        pl.col("DATE_").dt.truncate("1mo").alias("MONTH")
    ])
    .group_by(["REGION", "MONTH"])
    .agg([
        pl.sum("TOTALPRICE").alias("TOTAL_REVENUE")
    ])
    .sort(["REGION", "MONTH"])
)

home_monthly_pd = home_monthly.to_pandas()
home_monthly_pd["MONTH"] = pd.to_datetime(home_monthly_pd["MONTH"])
home_monthly_pd = home_monthly_pd.sort_values(["REGION", "MONTH"])

# ==========================
# 3️⃣ TẠO ĐẶC TRƯNG MẠNH HƠN
# ==========================
df_features = []

for region, group in home_monthly_pd.groupby("REGION"):
    group = group.copy()
    group = group.sort_values("MONTH")

    # Tạo các đặc trưng trễ (lag)
    for lag in [1, 2, 3, 6]:
        group[f"lag_{lag}"] = group["TOTAL_REVENUE"].shift(lag)

    # Trung bình trượt (rolling mean)
    group["rolling_mean_3"] = group["TOTAL_REVENUE"].rolling(3).mean()
    group["rolling_mean_6"] = group["TOTAL_REVENUE"].rolling(6).mean()

    # Xu hướng phần trăm tăng/giảm
    group["trend"] = group["TOTAL_REVENUE"].pct_change()

    # Đặc trưng thời gian
    group["month"] = group["MONTH"].dt.month
    group["year"] = group["MONTH"].dt.year

    # Mã hóa khu vực
    group["region_code"] = hash(region) % 1000

    df_features.append(group)

df_features = pd.concat(df_features).dropna()

# ==========================
# 4️⃣ LOẠI CỘT KHÔNG CÓ BIẾN THIÊN
# ==========================
constant_cols = [c for c in df_features.columns if df_features[c].nunique() <= 1]
if constant_cols:
    print("🧹 Loại bỏ các cột không có biến thiên:", constant_cols)
    df_features = df_features.drop(columns=constant_cols)

# ==========================
# 5️⃣ HUẤN LUYỆN MÔ HÌNH LIGHTGBM
# ==========================
X = df_features.drop(columns=["TOTAL_REVENUE", "REGION", "MONTH"])
y = df_features["TOTAL_REVENUE"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

model = LGBMRegressor(
    objective="regression",
    learning_rate=0.03,
    n_estimators=1000,
    num_leaves=64,
    max_depth=-1,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42
)

model.fit(X_train, y_train)

# ==========================
# 6️⃣ ĐÁNH GIÁ ĐỘ CHÍNH XÁC MÔ HÌNH
# ==========================
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\n🎯 ĐỘ CHÍNH XÁC MÔ HÌNH LIGHTGBM:")
print(f"✅ MAE  (Sai số tuyệt đối trung bình): {mae:,.0f}")
print(f"✅ RMSE (Sai số bình phương trung bình): {rmse:,.0f}")
print(f"✅ R²    (Hệ số xác định): {r2:.4f}")

# ==========================
# 7️⃣ DỰ BÁO 6 THÁNG TỚI
# ==========================
future_preds = []

for region, group in home_monthly_pd.groupby("REGION"):
    last = group.copy()
    for i in range(6):
        last_month = last["MONTH"].max()
        next_month = last_month + pd.DateOffset(months=1)

        lag_1 = last.iloc[-1]["TOTAL_REVENUE"]
        lag_2 = last.iloc[-2]["TOTAL_REVENUE"]
        lag_3 = last.iloc[-3]["TOTAL_REVENUE"]
        lag_6 = last.iloc[-6]["TOTAL_REVENUE"] if len(last) >= 6 else lag_3
        rolling_mean_3 = np.mean([lag_1, lag_2, lag_3])
        rolling_mean_6 = np.mean(last["TOTAL_REVENUE"].tail(6))
        trend = (lag_1 - lag_2) / lag_2 if lag_2 != 0 else 0

        X_future = pd.DataFrame([{
            "lag_1": lag_1,
            "lag_2": lag_2,
            "lag_3": lag_3,
            "lag_6": lag_6,
            "rolling_mean_3": rolling_mean_3,
            "rolling_mean_6": rolling_mean_6,
            "trend": trend,
            "month": next_month.month,
            "year": next_month.year,
            "region_code": hash(region) % 1000
        }])

        y_pred = model.predict(X_future)[0]

        new_row = {
            "REGION": region,
            "MONTH": next_month,
            "TOTAL_REVENUE": y_pred
        }

        last = pd.concat([last, pd.DataFrame([new_row])], ignore_index=True)
        future_preds.append(new_row)

future_df = pd.DataFrame(future_preds)

# ==========================
# 8️⃣ VẼ BIỂU ĐỒ DỰ BÁO
# ==========================
fig = px.line(
    pd.concat([home_monthly_pd, future_df]),
    x="MONTH",
    y="TOTAL_REVENUE",
    color="REGION",
    title="📈 Dự báo doanh thu danh mục Home theo khu vực (6 tháng tới)",
    markers=True
)

fig.update_layout(
    xaxis_title="Thời gian (Tháng)",
    yaxis_title="Tổng doanh thu (TOTALPRICE)",
    legend_title="Khu vực (REGION)",
    template="plotly_white"
)

fig.show()


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001006 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 374
[LightGBM] [Info] Number of data points in the train set: 145, number of used features: 10
[LightGBM] [Info] Start training from score 10391152.815517
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

In [19]:


# ==============================================================
# 📈 DỰ BÁO DOANH THU DANH MỤC HOME THEO KHU VỰC (6 THÁNG TỚI)
# ==============================================================
import polars as pl
import pandas as pd
import numpy as np
import plotly.express as px
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ==========================
# 1️⃣ LỌC DỮ LIỆU DANH MỤC HOME
# ==========================
name_df = (
    merged.lazy()
    .filter(pl.col("ITEMNAME") == "Musical Heart Plush Bear 37 Cm")
    .filter(pl.col("REGION") != "unknown")
    .select(["REGION", "DATE_", "TOTALPRICE"])
    .collect()
)

# ==========================
# 2️⃣ GỘP THEO THÁNG
# ==========================
name_monthly = (
    name_df
    .with_columns([
        pl.col("DATE_").dt.truncate("1mo").alias("MONTH")
    ])
    .group_by(["REGION", "MONTH"])
    .agg([
        pl.sum("TOTALPRICE").alias("TOTAL_REVENUE")
    ])
    .sort(["REGION", "MONTH"])
)

name_monthly_pd = name_monthly.to_pandas()
name_monthly_pd["MONTH"] = pd.to_datetime(name_monthly_pd["MONTH"])
name_monthly_pd = name_monthly_pd.sort_values(["REGION", "MONTH"])

# ==========================
# 3️⃣ TẠO ĐẶC TRƯNG MẠNH HƠN
# ==========================
df_features = []

for region, group in name_monthly_pd.groupby("REGION"):
    group = group.copy()
    group = group.sort_values("MONTH")

    # Tạo các đặc trưng trễ (lag)
    for lag in [1, 2, 3, 6]:
        group[f"lag_{lag}"] = group["TOTAL_REVENUE"].shift(lag)

    # Trung bình trượt (rolling mean)
    group["rolling_mean_3"] = group["TOTAL_REVENUE"].rolling(3).mean()
    group["rolling_mean_6"] = group["TOTAL_REVENUE"].rolling(6).mean()

    # Xu hướng phần trăm tăng/giảm
    group["trend"] = group["TOTAL_REVENUE"].pct_change()

    # Đặc trưng thời gian
    group["month"] = group["MONTH"].dt.month
    group["year"] = group["MONTH"].dt.year

    # Mã hóa khu vực
    group["region_code"] = hash(region) % 1000

    df_features.append(group)

df_features = pd.concat(df_features).dropna()

# ==========================
# 4️⃣ LOẠI CỘT KHÔNG CÓ BIẾN THIÊN
# ==========================
constant_cols = [c for c in df_features.columns if df_features[c].nunique() <= 1]
if constant_cols:
    print("🧹 Loại bỏ các cột không có biến thiên:", constant_cols)
    df_features = df_features.drop(columns=constant_cols)

# ==========================
# 5️⃣ HUẤN LUYỆN MÔ HÌNH LIGHTGBM
# ==========================
X = df_features.drop(columns=["TOTAL_REVENUE", "REGION", "MONTH"])
y = df_features["TOTAL_REVENUE"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

model = LGBMRegressor(
    objective="regression",
    learning_rate=0.03,
    n_estimators=1000,
    num_leaves=64,
    max_depth=-1,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42
)

model.fit(X_train, y_train)

# ==========================
# 6️⃣ ĐÁNH GIÁ ĐỘ CHÍNH XÁC MÔ HÌNH
# ==========================
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\n🎯 ĐỘ CHÍNH XÁC MÔ HÌNH LIGHTGBM:")
print(f"✅ MAE  (Sai số tuyệt đối trung bình): {mae:,.0f}")
print(f"✅ RMSE (Sai số bình phương trung bình): {rmse:,.0f}")
print(f"✅ R²    (Hệ số xác định): {r2:.4f}")

# ==========================
# 7️⃣ DỰ BÁO 6 THÁNG TỚI
# ==========================
future_preds = []

for region, group in name_monthly_pd.groupby("REGION"):
    last = group.copy()
    for i in range(6):
        last_month = last["MONTH"].max()
        next_month = last_month + pd.DateOffset(months=1)

        lag_1 = last.iloc[-1]["TOTAL_REVENUE"]
        lag_2 = last.iloc[-2]["TOTAL_REVENUE"]
        lag_3 = last.iloc[-3]["TOTAL_REVENUE"]
        lag_6 = last.iloc[-6]["TOTAL_REVENUE"] if len(last) >= 6 else lag_3
        rolling_mean_3 = np.mean([lag_1, lag_2, lag_3])
        rolling_mean_6 = np.mean(last["TOTAL_REVENUE"].tail(6))
        trend = (lag_1 - lag_2) / lag_2 if lag_2 != 0 else 0

        X_future = pd.DataFrame([{
            "lag_1": lag_1,
            "lag_2": lag_2,
            "lag_3": lag_3,
            "lag_6": lag_6,
            "rolling_mean_3": rolling_mean_3,
            "rolling_mean_6": rolling_mean_6,
            "trend": trend,
            "month": next_month.month,
            "year": next_month.year,
            "region_code": hash(region) % 1000
        }])

        y_pred = model.predict(X_future)[0]

        new_row = {
            "REGION": region,
            "MONTH": next_month,
            "TOTAL_REVENUE": y_pred
        }

        last = pd.concat([last, pd.DataFrame([new_row])], ignore_index=True)
        future_preds.append(new_row)

future_df = pd.DataFrame(future_preds)

# ==========================
# 8️⃣ VẼ BIỂU ĐỒ DỰ BÁO
# ==========================
fig = px.line(
    pd.concat([name_monthly_pd, future_df]),
    x="MONTH",
    y="TOTAL_REVENUE",
    color="REGION",
    title="📈 Dự báo doanh thu danh mục Musical Heart Plush Bear 37 Cm theo khu vực (6 tháng tới)",
    markers=True
)

fig.update_layout(
    xaxis_title="Thời gian (Tháng)",
    yaxis_title="Tổng doanh thu (TOTALPRICE)",
    legend_title="Khu vực (REGION)",
    template="plotly_white"
)

fig.show()


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000048 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 374
[LightGBM] [Info] Number of data points in the train set: 145, number of used features: 10
[LightGBM] [Info] Start training from score 55379.547047
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain